In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/caltech-101/

/gdrive/MyDrive/caltech-101


In [ ]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
image_paths = []
for class_name in os.listdir("./data"):
    class_path = os.path.join("./data", class_name)
    for image_name in os.listdir(class_path):
        image_paths.append((os.path.join(class_path, image_name), class_name))

In [ ]:
len(image_paths)

453

In [ ]:
# Split data into train and test sets
train_paths, test_paths = train_test_split(image_paths, test_size=0.05, random_state=42)

In [ ]:
print(len(train_paths))
print(len(test_paths))

430
23


In [ ]:
def get_descriptors(sift, img):
    kp, des = sift.detectAndCompute(img, None)
    return des

def read_image(img_path):
    img = cv2.imread(img_path, 0)
    return cv2.resize(img, (150, 150))

def extract_features(image_paths, sift):
    descriptor_list = []
    labels = []
    for img_path, label in image_paths:
        img = read_image(img_path)
        des = get_descriptors(sift, img)
        if des is not None:
            descriptor_list.append(des)
            labels.append(label)
    return descriptor_list, labels

In [ ]:
# Initialize SIFT
sift = cv2.SIFT_create()

# Extract features from train set
train_descriptors, train_labels = extract_features(train_paths, sift)

In [ ]:
def stack_descriptors(descriptor_list):
    descriptors = np.array(descriptor_list[0])
    for descriptor in descriptor_list[1:]:
        descriptors = np.vstack((descriptors, descriptor))
    return descriptors
train_descriptors_stacked = stack_descriptors(train_descriptors)

In [ ]:
num_clusters = 100
kmeans = KMeans(n_clusters=num_clusters).fit(train_descriptors_stacked)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
def extract_image_features(descriptor_list, kmeans, num_clusters):
    image_features = np.zeros((len(descriptor_list), num_clusters))
    for i, descriptors in enumerate(descriptor_list):
        for descriptor in descriptors:
            idx = kmeans.predict(descriptor.reshape(1, -1))
            image_features[i][idx] += 1
    return image_features
# Extract image features
train_features = extract_image_features(train_descriptors, kmeans, num_clusters)

# Normalize features

train_features_normalized = scaler.fit_transform(train_features)

In [ ]:
# Train SVM
def train_svm(X_train, y_train, kernel='linear'):
    svm = SVC(kernel=kernel)
    svm.fit(X_train, y_train)
    return svm
svm = train_svm(train_features_normalized, train_labels)

In [ ]:
# Extract features from test set
test_descriptors, test_labels = extract_features(test_paths, sift)
test_features = extract_image_features(test_descriptors, kmeans, num_clusters)
test_features_normalized = scaler.transform(test_features)

# Test SVM
predictions = svm.predict(test_features_normalized)

# Evaluate
accuracy = np.mean(predictions == test_labels)
print("Accuracy:", accuracy)

Accuracy: 0.391304347826087


In [ ]:
num_clusters = 400
kmeans = KMeans(n_clusters=num_clusters).fit(train_descriptors_stacked)

# Extract image features
train_features = extract_image_features(train_descriptors, kmeans, num_clusters)

# Normalize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)

svm = train_svm(train_features_normalized, train_labels)

# Extract features from test set
test_descriptors, test_labels = extract_features(test_paths, sift)
test_features = extract_image_features(test_descriptors, kmeans, num_clusters)
test_features_normalized = scaler.transform(test_features)

# Test SVM
predictions = svm.predict(test_features_normalized)

# Evaluate
accuracy = np.mean(predictions == test_labels)
print("Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy: 0.5652173913043478
